In [ ]:
#Type1_possession
import pandas as pd
import random
import openai

def generate_sentence_with_gpt(subject, pronoun, adjective, possession, verb, complement, possession_2_flag):
    examples = (
        "If John has a young daughter, he'll bring his young daughter to the party.\n"
        "If Melissa has an older son, she'll furnish her older son's house.\n"
        "If Bob and Mary have a young daughter, they'll feed their young daughter.\n"
    )

    prompt = (
        f"Generate a sentence in the form:\n"
        f"If [subject] has [adjective] [possession], they'll [verb] [adjective] [possession] [complement].\n\n"
        f"Here are some examples:\n"
        f"{examples}\n"
        f"Now generate a sentence with:\n"
        f"Subject: {subject}\n"
        f"Pronoun: {pronoun}\n"
        f"Adjective: {adjective}\n"
        f"Possession: {possession}\n"
        f"Verb: {verb}\n"
        f"Complement: {complement}\n"
        f"Possession's Flag: {possession_2_flag}\n"
    )

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant generating structured sentences."},
            {"role": "user", "content": prompt},
        ],
        max_tokens=50,
        temperature=0.7,
    )

    sentence = response.choices[0].message.content.strip()
    return sentence


def main():
    # load data from the csv file
    file_path = "/content/Possession_Type1_Final.csv"
    data = pd.read_csv(file_path)



    data = data.fillna(0)

    # mapping markers to corresponding words in the vocabulary column
    subjects = data[data['subject'] == 1]['vocabulary'].tolist()
    verbs = data[data['verb'] == 1]['vocabulary'].tolist()
    possessions = data[data['possession'] == 1]['vocabulary'].tolist()
    adjectives = data[data['adjective'] == 1]['vocabulary'].tolist()
    complements = data[data['cmp'] != 0][['vocabulary', 'cmp']].dropna()
    possession_2_flags = data.set_index('vocabulary')['possession_2'].fillna(0).to_dict()


    pronouns = {
    sub: "their" if "and" in sub or "," in sub else "her" if sub in ["Alice", "Mary", "Melissa", "The queen", "Nadia"] else "his"
    for sub in subjects
    }

    sentences_generated = set()


    while len(sentences_generated) < 50:
        subject = random.choice(subjects)
        pronoun = pronouns[subject]
        verb = random.choice(verbs)
        possession = random.choice(possessions)
        adjective = random.choice(adjectives)


        if complement_values := complements[complements['vocabulary'] == verb]['cmp'].tolist():
            complement = random.choice(complement_values[0].split(", "))
        else:
            complement = None

        possession_2_flag = possession_2_flags.get(verb, 0)


        if adjective and possession:
            possession_phrase = f"{adjective} {possession}"
        else:
            possession_phrase = possession

        sentence = generate_sentence_with_gpt(
            subject, pronoun, adjective, possession_phrase, verb, complement, possession_2_flag
        )
        sentences_generated.add(sentence)


    for sentence in sentences_generated:
        print(sentence)

if __name__ == "__main__":
    main()


If Jennifer has a patient best friend, he'll tell a secret.
If the women have a talented partner, they'll start his talented partner in a business.
If the guy has a supportive brother, he'll look at his supportive brother with love.
If the guy has a dutiful close friend, he'll learn from his dutiful close friend.
If Mary has a compassionate uncle, she'll serve her compassionate uncle hot tea.
If the boy has a supportive nephew, he'll dictate his supportive nephew a new lifestyle.
If Joel has a compassionate stepsister, he'll reach out to his compassionate stepsister.
If the lady has a friendly son-in-law, she'll offer his friendly son-in-law tea.
If the policeman has a kind cousin, he'll dance with his kind cousin.
If Joel has a friendly stepsister, he'll put his friendly stepsister in an awkward situation.
If Mary and Bob have an empathetic granddaughter, they'll cooperate with their empathetic granddaughter.
If Curtis has a charismatic close friend, he'll explain his charismatic clos

In [ ]:
#Type1_again
import pandas as pd
import openai
import random


def generate_sentence_with_gpt(verb, instance, subject):
    """
    Generates a sentence using GPT based on structured input with verbs, instances, and subjects.
    """

    prompt = (
        f"Generate a sentence in the form:\n"
        f"If [subject] [verb in past tense] [instance], they'll never [verb] [instance] again.\n"
        f"For example:\n"
        f"If he learned a new skill, he'll never learn a new skill again.\n"
        f"If Alice watched a movie by Nolan, she'll never watch a movie by Nolan again.\n\n"
        f"Now generate a sentence with:\n"
        f"Subject: {subject}\n"
        f"Verb: {verb} (in past tense)\n"
        f"Instance: {instance}\n"
    )


    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant generating structured sentences."},
            {"role": "user", "content": prompt},
        ],
        max_tokens=50,
        temperature=0.7,
    )


    sentence = response.choices[0].message.content.strip()
    return sentence


def main():

    file_path = "/content/drive/MyDrive/Colab Notebooks/cleaned_type1Test.csv"
    data = pd.read_csv(file_path)

    # extract verbs (vocabulary column) and their hyponyms
    verbs = data['verbs'].tolist()
    hyponyms_list = data['instance'].tolist()

    # map verbs to lists of hyponyms
    verb_hyponyms = {verb: hyponyms.split(', ') for verb, hyponyms in zip(verbs, hyponyms_list)}

    sentences_generated = []


    unique_subjects = ["John", "Alice", "Bob", "Melissa", "Nadia", "Carlos", "Angela", "Kimberley", "Ryan", "Wendy", "the boy", "Benjamin", "Joel", "the man", "Bruce", "Martin", "the old man", "the king", "the queen",
                       "Walter", "Curtis", "the senator", "Dan", "Jennifer", "Lisa", "Kimberley", "Michelle", "Amy", "Angela",
                       "Tammy", "Wendy", "the boss", "the president", "the professor", "the waiter", "the merchant", "the policeman", "the doctor", "the girl", "the woman", "the teacher", "the women", "the actor", "Marley", "Nadia", "the guy"]
    pronouns = ["he", "she", "they"]


    for verb, hyponyms in verb_hyponyms.items():
        for hyponym in hyponyms:
            # randomly choose a subject or pronoun
            subject = random.choice(unique_subjects + pronouns)
            try:
                sentence = generate_sentence_with_gpt(verb, hyponym, subject)
                sentences_generated.append({"Verb": verb, "Instance": hyponym, "Sentence": sentence})
                print(sentence)
                if len(sentences_generated) >= 400:
                    break
            except Exception as e:
                print(f"Error generating sentence for Verb: {verb}, Instance: {hyponym}. Error: {e}")
                continue
        if len(sentences_generated) >= 400:
            break


    output_file = "/content/generated_sentences_past_tense_type1.csv"
    pd.DataFrame(sentences_generated).to_csv(output_file, index=False)
    print(f"Sentences saved to {output_file}")


if __name__ == "__main__":
    main()


If the teacher watched a movie by Nolan, they'll never watch a movie by Nolan again.
If the policeman read a book by Dan Brown, he'll never read a book by Dan Brown again.
If Melissa ate a traditional Swedish dish, she'll never eat a traditional Swedish dish again.
If the old man sang a pop song, he'll never sing a pop song again.
If Jennifer listened to a pop song, she'll never listen to a pop song again.
If Angela wrote a horror novel, she'll never write a horror novel again.
If Lisa ran a marathon in the US, she'll never run a marathon in the US again.
If Kimberley painted a portrait, she'll never paint a portrait again.
If Dan drove a sports car, he'll never drive a sports car again.
If Ryan played a musical instrument at the talent show, he'll never play a musical instrument at a public event again.
If the queen cooked a chicken, she'll never cook a chicken again.
If the boy learned a fact about Tara, he'll never learn a fact about Tara again.
If Angela performed a dance show, she

In [ ]:
#Type2_possession
import pandas as pd
import random
import openai


def generate_sentence_with_gpt(subject, pronoun, adjective, possession, verb, complement, possession_2_flag):
    examples = (
        "If John has a young daughter, he'll bring his young daughter to the party.\n"
        "If Melissa has an older son, she'll furnish her older son's house.\n"
        "If Bob and Mary have a young daughter, they'll feed their young daughter.\n"
    )

    prompt = (
        f"Generate a sentence in the form:\n"
        f"If [subject] has [adjective] [possession], they'll [verb] [adjective] [possession] [complement].\n\n"
        f"Here are some examples:\n"
        f"{examples}\n"
        f"Now generate a sentence with:\n"
        f"Subject: {subject}\n"
        f"Pronoun: {pronoun}\n"
        f"Adjective: {adjective}\n"
        f"Possession: {possession}\n"
        f"Verb: {verb}\n"
        f"Complement: {complement}\n"
        f"Possession's Flag: {possession_2_flag}\n"
    )

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant generating structured sentences."},
            {"role": "user", "content": prompt},
        ],
        max_tokens=50,
        temperature=0.7,
    )

    sentence = response.choices[0].message.content.strip()
    return sentence


def main():

    file_path = "/content/Possession_Type1_Final.csv"
    data = pd.read_csv(file_path)



    data = data.fillna(0)


    subjects = data[data['subject'] == 1]['vocabulary'].tolist()
    verbs = data[data['verb'] == 1]['vocabulary'].tolist()
    possessions = data[data['possession'] == 1]['vocabulary'].tolist()
    adjectives = data[data['adjective'] == 1]['vocabulary'].tolist()
    complements = data[data['cmp'] != 0][['vocabulary', 'cmp']].dropna()
    possession_2_flags = data.set_index('vocabulary')['possession_2'].fillna(0).to_dict()



    pronouns = {
        sub: "their" if "and" in sub or "," in sub else "her" if sub in ["Alice", "Mary", "Melissa", "The queen", "Nadia"] else "his"
        for sub in subjects
    }

    sentences_generated = set()


    while len(sentences_generated) < 50:
        subject = random.choice(subjects)
        pronoun = pronouns[subject]
        verb = random.choice(verbs)
        possession = random.choice(possessions)
        adjective = random.choice(adjectives)


        if complement_values := complements[complements['vocabulary'] == verb]['cmp'].tolist():
            complement = random.choice(complement_values[0].split(", "))
        else:
            complement = None

        possession_2_flag = possession_2_flags.get(verb, 0)


        if adjective and possession:
            possession_phrase = f"{adjective} {possession}"
        else:
            possession_phrase = possession

        sentence = generate_sentence_with_gpt(
            subject, pronoun, adjective, possession_phrase, verb, complement, possession_2_flag
        )
        sentences_generated.add(sentence)


    for sentence in sentences_generated:
        print(sentence)

if __name__ == "__main__":
    main()


If Jennifer has a patient best friend, he'll tell a secret.
If the women have a talented partner, they'll start his talented partner in a business.
If the guy has a supportive brother, he'll look at his supportive brother with love.
If the guy has a dutiful close friend, he'll learn from his dutiful close friend.
If Mary has a compassionate uncle, she'll serve her compassionate uncle hot tea.
If the boy has a supportive nephew, he'll dictate his supportive nephew a new lifestyle.
If Joel has a compassionate stepsister, he'll reach out to his compassionate stepsister.
If the lady has a friendly son-in-law, she'll offer his friendly son-in-law tea.
If the policeman has a kind cousin, he'll dance with his kind cousin.
If Joel has a friendly stepsister, he'll put his friendly stepsister in an awkward situation.
If Mary and Bob have an empathetic granddaughter, they'll cooperate with their empathetic granddaughter.
If Curtis has a charismatic close friend, he'll explain his charismatic clos

In [ ]:
#Type2_again
import pandas as pd
import openai
import random


def generate_sentence_with_gpt(verb, instance, subject):
    """
    Generates a sentence using GPT based on structured input with verbs, instances, and subjects.
    """

    prompt = (
        f"Generate a sentence in the form:\n"
        f"If [subject] [verb in present tense] this [instance], they'll never [verb] [instance] again.\n"
        f"For example:\n"
        f"If he learns this new skill, he'll never learn a new skill again.\n"
        f"If you watch this movie by Nolan, you'll never watch a movie by Nolan again.\n\n"
        f"Now generate a sentence with:\n"
        f"Subject: {subject}\n"
        f"Verb: {verb} (in present tense)\n"
        f"Instance: this {instance}\n"
    )


    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant generating structured sentences."},
            {"role": "user", "content": prompt},
        ],
        max_tokens=50,
        temperature=0.7,
    )

    # extract and validate the generated sentence for "this" inclusion
    sentence = response.choices[0].message.content.strip()
    if "this" not in sentence.split()[3:6]:
        sentence = sentence.replace(" a ", " this ", 1)
    return sentence


def main():
    file_path = "/content/drive/MyDrive/Colab Notebooks/cleaned_type1Test.csv"
    data = pd.read_csv(file_path)

    verbs = data['verbs'].tolist()
    hyponyms_list = data['instance'].tolist()


    verb_hyponyms = {verb: hyponyms.split(', ') for verb, hyponyms in zip(verbs, hyponyms_list)}

    sentences_generated = []

    # list of unique names for subjects
    unique_subjects = ["John", "Alice", "Bob", "Melissa", "Nadia", "Carlos", "Angela", "Kimberley", "Ryan", "Wendy", "the boy", "Benjamin", "Joel", "the man", "Bruce", "Martin", "the old man", "the king", "the queen",
                       "Walter", "Curtis", "the senator", "Dan", "Jennifer", "Lisa", "Kimberley", "Michelle", "Amy", "Angela",
                       "Tammy", "Wendy", "the boss", "the president", "the professor", "the waiter", "the merchant", "the policeman", "the doctor", "the girl", "the woman", "the teacher", "the women", "the actor", "Marley", "Nadia", "the guy"]
    pronouns = ["he", "she", "they"]


    for verb, hyponyms in verb_hyponyms.items():
        for hyponym in hyponyms:
            # randomly choose a subject or pronoun
            subject = random.choice(unique_subjects + pronouns)
            try:
                sentence = generate_sentence_with_gpt(verb, hyponym, subject)
                sentences_generated.append({"Verb": verb, "Instance": hyponym, "Sentence": sentence})
                print(sentence)
                if len(sentences_generated) >= 400:
                    break
            except Exception as e:
                print(f"Error generating sentence for Verb: {verb}, Instance: {hyponym}. Error: {e}")
                continue
        if len(sentences_generated) >= 400:
            break


    output_file = "/content/generated_sentences_with_this.csv"
    pd.DataFrame(sentences_generated).to_csv(output_file, index=False)
    print(f"Sentences saved to {output_file}")


if __name__ == "__main__":
    main()


If the president watches this movie by Nolan, he'll never watch a movie by Nolan again.
If the teacher reads this book by Dan Brown, they'll never read a book by Dan Brown again.
If the boy eats this traditional Swedish dish, he'll never eat a traditional Swedish dish again.
If Tammy sings this pop song, she'll never sing a pop song again.
If Martin listens to this pop song, he'll never listen to a pop song again.
If the merchant writes this horror novel, they'll never write a horror novel again.
If the merchant runs this marathon in the US, they'll never run a marathon in the US again.
If Melissa paints this portrait, she'll never paint a portrait again.
If the policeman drives this sports car, he'll never drive a sports car again.
If Amy plays this musical instrument, she'll never play a musical instrument again.
If the girl cooks this chicken, she'll never cook a chicken again.
If Kimberley learns this fact about Tara, she'll never learn it again.
If the girl performs in this dance 

In [ ]:
#Type3_again
import pandas as pd
import openai
import random

def generate_sentence_with_gpt(subject, verb, noun, instance, pronoun):
    """
    Generates a sentence using GPT based on structured input with verbs, nouns, instances, and subjects.
    """

    examples = (
        "If she ever watched a movie, she didn't watch Star Wars again.\n"
        "If he ever read a book, he didn't read Harry Potter again.\n"
        "If Alice ever listened to a symphony, she didn't listen to Beethoven's 5th again.\n"
    )

    prompt = (
        f"Generate a sentence in the form:\n"
        f"If [subject] ever [verb in past tense] a [noun], [pronoun] didn't [verb] [instance] again.\n\n"
        f"Here are some examples:\n"
        f"{examples}\n"
        f"Now generate a sentence with:\n"
        f"Subject: {subject}\n"
        f"Verb: {verb} (in past tense)\n"
        f"Noun: {noun}\n"
        f"Instance: {instance}\n"
        f"Pronoun: {pronoun}\n"

    )


    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant generating structured sentences."},
            {"role": "user", "content": prompt},
        ],
        max_tokens=50,
        temperature=0.7,
    )


    sentence = response.choices[0].message.content.strip()
    return sentence


def main():

    file_path = "/content/drive/MyDrive/Colab Notebooks/type3-updated-EverAgainData.csv"
    data = pd.read_csv(file_path)


    verbs = data['verb'].tolist()
    nouns = data['noun'].tolist()
    instances = data['instance'].tolist()

    # unique subjects and pronouns
    unique_subjects = [
        "John", "Alice", "Bob", "Melissa", "Nadia", "Carlos", "Angela", "Kimberley", "Ryan", "Wendy",
        "the boy", "Benjamin", "Joel", "the man", "Bruce", "Martin", "the old man", "the king", "the queen",
        "Walter", "Curtis", "the senator", "Dan", "Jennifer", "Lisa", "Michelle", "Amy", "Angela",
        "Tammy", "Wendy", "the boss", "the president", "the professor", "the waiter", "the merchant",
        "the policeman", "the doctor", "the girl", "the woman", "the teacher", "the actor", "Marley",
        "Nadia", "the guy"
    ]
    pronouns = {
        "John": "he", "Bob": "he", "Ryan": "he", "Carlos": "he", "Benjamin": "he", "Joel": "he", "Bruce": "he",
        "Martin": "he", "Walter": "he", "Curtis": "he", "Dan": "he", "the boy": "he", "the man": "he",
        "the old man": "he", "the king": "he", "the senator": "he", "the boss": "he", "the professor": "he",
        "the waiter": "he", "the merchant": "he", "the policeman": "he", "the doctor": "he", "the actor": "he",
        "Alice": "she", "Melissa": "she", "Nadia": "she", "Angela": "she", "Kimberley": "she", "Wendy": "she",
        "Lisa": "she", "Michelle": "she", "Amy": "she", "Tammy": "she", "Jennifer": "she", "the girl": "she",
        "the woman": "she", "the teacher": "she", "the queen": "she"
    }

    # track used subjects to maintain uniqueness
    used_subjects = set()


    sentences_generated = []
    while len(sentences_generated) < 400:
        for verb, noun, instance in zip(verbs, nouns, instances):
            remaining_subjects = [sub for sub in unique_subjects if sub not in used_subjects]
            if not remaining_subjects:
                used_subjects.clear()
                remaining_subjects = unique_subjects

            subject = random.choice(remaining_subjects)
            used_subjects.add(subject)
            pronoun = pronouns.get(subject, "they")

            try:
                sentence = generate_sentence_with_gpt(subject, verb, noun, instance, pronoun)
                if sentence not in sentences_generated:
                    sentences_generated.append(sentence)
                    print(f"Sentence: {sentence}")
            except Exception as e:
                print(f"Error generating sentence for {subject}, {verb}, {noun}: {e}")
                continue

            if len(sentences_generated) >= 400:
                break
        if len(sentences_generated) >= 400:
            break


    output_file = "/content/type3-generated-sentences.csv"
    pd.DataFrame({"Sentence": sentences_generated}).to_csv(output_file, index=False)
    print(f"Sentences saved to {output_file}")


if __name__ == "__main__":
    main()


Sentence: If Ryan ever watched a movie, he didn't watch Star Wars again.
Sentence: If the boss ever read a book, he didn't read Harry Potter again.
Sentence: If Angela ever watched a TV show, she didn't watch Friends again.
Sentence: If the teacher ever sang a song, she didn't sing Bohemian Rhapsody again.
Sentence: If the senator ever admired an artist, he didn't admire Leonardo da Vinci again.
Sentence: If the queen ever liked an album, she didn't listen to Thriller again.
Sentence: If Nadia ever followed an actor, she didn't follow Leonardo DiCaprio again.
Sentence: If Alice ever followed an actress, she didn't follow Meryl Streep again.
Sentence: If the man ever admired a Director, he didn't admire Steven Spielberg again.
Sentence: If Wendy ever listened to a band, she didn't listen to The Beatles again.
Sentence: If Dan ever admired a painting, he didn't admire the Mona Lisa again.
Sentence: If the actor ever listened to a symphony, he didn't listen to Beethoven's 5th again.
Sente

In [ ]:
#Type3_possession
import pandas as pd
import openai
import random

def generate_sentence_with_gpt(subject, hypernym, hyponym, ownership_verb, pronoun):
    """
    Generates a sentence using GPT based on structured input.
    Args:
        subject: The subject of the sentence (e.g., "John").
        hypernym: The category (e.g., "children").
        hyponym: A specific instance (e.g., "4-year-old daughter").
        ownership_verb: The verb indicating possession (e.g., "has", "owns", "purchased").
        pronoun: The pronoun corresponding to the subject (e.g., "his", "her", "their").
    Returns:
        Generated sentence as a string.
    """

    prompt = (
        f"Generate a sentence in the form:\n"
        f"If [subject] [ownership_verb] [hypernym], [pronoun] will [verb] [pronoun] [hyponym].\n"
        f"For example:\n"
        f"If John has children, he'll bring his four-year-old daughter to the party.\n"
        f"If the king owns children, he'll kiss his sons.\n\n"
        f"Now generate a sentence with:\n"
        f"Subject: {subject}\n"
        f"Ownership Verb: {ownership_verb}\n"
        f"Hypernym: {hypernym}\n"
        f"Hyponym: {hyponym}\n"
        f"Pronoun: {pronoun}\n"
    )


    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant generating structured sentences."},
            {"role": "user", "content": prompt},
        ],
        max_tokens=50,
        temperature=0.7,
    )


    sentence = response.choices[0].message.content.strip()
    return sentence


def main():
    file_path = "/content/drive/MyDrive/Colab Notebooks/Type3-Final-Possession.csv"
    data = pd.read_csv(file_path)

    # Extract subjects, hypernyms, and their corresponding hyponyms
    subjects = data[data['subject'] == 1]['vocabulary'].tolist()
    hypernyms = data[data['hypernym'] == 1]['vocabulary'].tolist()
    hyponyms_data = data[data['hyponym'] != ""][['vocabulary', 'hyponym']].dropna()

    # map hypernyms to lists of hyponyms
    hypernym_hyponyms = {
        row['vocabulary']: row['hyponym'].split(', ') for _, row in hyponyms_data.iterrows()
    }

    # generate possessive pronouns for subjects where necessary
    pronouns = {
        sub: "their" if "and" in sub or "," in sub else "her" if sub in ["Alice", "Mary", "Melissa", "The queen"] else "his"
        for sub in subjects
    }


    sentences_generated = []
    ownership_verbs = ["has"]

    while len(sentences_generated) < 400:
        for hypernym, hyponyms in hypernym_hyponyms.items():
            for hyponym in hyponyms:
                # randomly select a subject and its corresponding pronoun
                subject = random.choice(subjects)
                pronoun = pronouns[subject]

                # randomly select an ownership verb
                ownership_verb = random.choice(ownership_verbs)

                try:
                    sentence = generate_sentence_with_gpt(subject, hypernym, hyponym, ownership_verb, pronoun)
                    sentences_generated.append({"Subject": subject, "Hypernym": hypernym, "Hyponym": hyponym, "Sentence": sentence})
                    print(f"Sentence: {sentence}")
                    if len(sentences_generated) >= 400:
                        break
                except Exception as e:
                    print(f"Error generating sentence for Subject: {subject}, Hypernym: {hypernym}, Hyponym: {hyponym}. Error: {e}")
                    continue

            if len(sentences_generated) >= 400:
                break
        if len(sentences_generated) >= 400:
            break


    output_file = "/content/generated_sentences.csv"
    pd.DataFrame(sentences_generated).to_csv(output_file, index=False)
    print(f"Generated sentences saved to {output_file}")

if __name__ == "__main__":
    main()

Sentence: If Jason has children, he'll take his son to the baseball game.
Sentence: If Lucille has children, she'll bring his daughter to the school play.
Sentence: If Craig has children, he'll teach his sons how to play baseball.
Sentence: If Timothy has children, he'll take his daughters to the dance recital.
Sentence: If Alexander has children, they will bring their 4-year-old daughter to the playground.
Sentence: If Carmen has children, she will take his curious son to the science museum.
Sentence: If Randolf has children, their youngest son will accompany them to the family gathering.
Sentence: If Anna has children, she'll discipline his naughty daughter.
Sentence: If Rhonda has pets, she will stroke his pet cat.
Sentence: If Theresa has pets, she will walk his pet dog in the park.
Sentence: If Amy has pets, she will feed his pet bird.
Sentence: If Heidi has pets, she will cuddle his pet rabbit.
Sentence: If James has cars, he will drive his red car to the car show.
Sentence: If A

In [ ]:
#Type4_possession
def generate_sentence_with_gpt(subject, obj, verb2, adverb):
    """
    Generates a sentence using GPT based on a structured prompt.
    Args:
        subject: The subject of the sentence (e.g., "John").
        obj: The object (e.g., "bird").
        verb2: The second verb (e.g., "flies").
        adverb: The adverb describing the action (e.g., "high").
    Returns:
        Generated sentence as a string.
    """
    examples = (
        "If Mary has a bird, I wonder how high it flies.\n"
        "If John has a car, I wonder how fast it goes.\n"
        "If Alice has a piano, I wonder how beautifully it plays.\n"
    )

    prompt = (
        f"Generate a sentence in the form:\n"
        f"If [subject] has a [object], I wonder how [adverb] it [verb2].\n\n"
        f"Here are some examples:\n"
        f"{examples}\n"
        f"Now generate a sentence with:\n"
        f"Subject: {subject}\n"
        f"Object: {obj}\n"
        f"Verb2: {verb2}\n"
        f"Adverb: {adverb}\n"
    )

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant generating structured sentences."},
            {"role": "user", "content": prompt},
        ],
        max_tokens=50,
        temperature=0.7,
    )

    sentence = response.choices[0].message.content.strip()
    return sentence


def main():
    file_path = "/content/drive/MyDrive/Colab Notebooks/Type4-possessive-FinalData.csv"
    data = pd.read_csv(file_path)


    subjects = data[data['subjects'] == 1]['vocabulary'].tolist()
    objects = data[data['object'] == 1]['vocabulary'].tolist()
    verb2_adverbs = data[['vocabulary', 'verb2', 'adverb']].dropna()

    sentences_generated = []


    while len(sentences_generated) < 292:
        for _, row in verb2_adverbs.iterrows():
            subject = random.choice(subjects)
            obj = row['vocabulary']
            verb2 = row['verb2']
            adverb = row['adverb']

            try:
                sentence = generate_sentence_with_gpt(subject, obj, verb2, adverb)
                sentences_generated.append(sentence)
                print(f"Sentence: {sentence}")


                if len(sentences_generated) >= 292:
                    break
            except Exception as e:
                print(f"Error generating sentence for {subject}, {obj}, {verb2}: {e}")
                continue

        if len(sentences_generated) >= 292:
            break


    output_file = "/content/Type4-possessive-generated-sentences.csv"
    pd.DataFrame({"Sentence": sentences_generated}).to_csv(output_file, index=False)
    print(f"Sentences saved to {output_file}")

if __name__ == "__main__":
    main()


Sentence: If the cashier has a car, I wonder how fast it goes.
Sentence: If the customer has a cat, I wonder how fast it runs.
Sentence: If Maria has a dog, I wonder how fast it runs.
Sentence: If Sheila has a bird, I wonder how high it flies.
Sentence: If Becky has a guitar, I wonder how smooth it sounds.
Sentence: If David has a camera, I wonder how well it captures moments.
Sentence: If Angela has a drone, I wonder how high it flies.
Sentence: If Elizabeth has a bird, I wonder how happily it chirps.
Sentence: If Keith has a guitar, I wonder how melodically it strums.
Sentence: If Kendra has a computer, I wonder how quickly it processes information.
Sentence: If Melinda has a piano, I wonder how beautifully it plays.
Sentence: If Roger has a student, I wonder how diligently it studies.
Sentence: If Dana has an experienced chef, I wonder how expertly they cook.
Sentence: If Holly has a horse, I wonder how gracefully it trots.
Sentence: If Aaron has a train, I wonder how fast it travel

In [ ]:
#Type4_occupation_tool_possession
def generate_sentence_with_gpt(subject, occupation, tool):
    """
    Generates a sentence using GPT based on the specified structure.
    """

    prompt = (
        f"Generate a sentence in the form:\n"
        f"If [subject] is a [occupation], they'll [verb] their [tool].\n"
        f"For example:\n"
        f"If John is a scuba diver, he'll bring his wetsuit.\n"
        f"If Melissa is a doctor, she'll use her stethoscope.\n\n"
        f"Now generate a sentence with:\n"
        f"Subject: {subject}\n"
        f"Occupation: {occupation}\n"
        f"Tool: {tool}\n"
        f"Choose an appropriate verb based on the tool.\n"
    )


    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant generating structured sentences."},
            {"role": "user", "content": prompt},
        ],
        max_tokens=50,
        temperature=0.7,
    )

    sentence = response.choices[0].message.content.strip()
    return sentence


def main():
    file_path = "/content/drive/MyDrive/Colab Notebooks/Final-Type4-OccupationData.csv"
    data = pd.read_csv(file_path)


    subjects = data[data['subject'] == 1]['vocabulary'].tolist()
    occupations = data[data['occupation'] == 1]['vocabulary'].tolist()
    tools = data[data['tool'].notna()]['tool'].tolist()

    random.shuffle(subjects)
    random.shuffle(occupations)
    random.shuffle(tools)

    sentences_generated = []
    used_subjects = set()


    while len(sentences_generated) < 400:
        if len(used_subjects) < len(subjects):
            subject = next(sub for sub in subjects if sub not in used_subjects)
            used_subjects.add(subject)
        else:
          subject = random.choice(subjects)

        # randomly choose occupation and tool for each subject
        occupation = random.choice(occupations)
        tool = random.choice(tools)

        try:
            sentence = generate_sentence_with_gpt(subject, occupation, tool)
            sentences_generated.append({"Subject": subject, "Occupation": occupation, "Tool": tool, "Sentence": sentence})
            print(sentence)
        except Exception as e:
            print(f"Error generating sentence for {subject}, {occupation}, {tool}: {e}")
            continue


    output_file = "/content/generated_sentences_occupation_data.csv"
    pd.DataFrame(sentences_generated).to_csv(output_file, index=False)
    print(f"Sentences saved to {output_file}")


if __name__ == "__main__":
    main()


If the cashier is a tour guide, they'll utilize their research database.
If Gerald is a sports coach, he'll measure athletes' calorie expenditure with his calorimeter.
If Jacqueline is a dancer, she'll follow her brand guidelines.
If Amy is a life coach, she'll have her first aid kit ready to assist in any emergencies.
If Sharon is an arborist, she'll ride her surfboard.
If Christopher is a silviculturist, he'll analyze data on his computer.
If Timothy is a graffiti artist, he'll organize his street art projects using event planning software.
If Travis is a photographer, he'll set up his anvil to use as a makeshift tripod for stability.
If the lady is an automotive engineer, she'll cultivate her greenhouse supplies.
If Colleen is a paralegal, she'll navigate with her trailblazer.
If the guy is a food scientist, he'll tighten bolts with his wrench.
If Margaret is a cartoonist, she'll sketch her drawings on her tablet.
If Frank is a printmaker, he'll organize his projects using schedulin

In [ ]:
#Type5_possession
import pandas as pd
import openai
import random

def generate_sentence_with_gpt(subject, verb1, cmp1, second_clause):
    """
    Generates a sentence using GPT based on a structured prompt.
    Args:
        subject: The subject of the sentence (e.g., "John").
        verb1: The first verb (e.g., "flies").
        cmp1: The complement for the first verb (e.g., "to Toronto"), or None if not required.
        second_clause: The second clause of the sentence.
    Returns:
        Generated sentence as a string.
    """

    if cmp1 == "0":
        sentence_start = f"If {subject} {verb1}, {second_clause}"
    else:
        sentence_start = f"If {subject} {verb1} {cmp1}, {second_clause}"


    prompt = (
        f"Generate a sentence in the form:\n"
        f"{sentence_start}\n"
        f"For example:\n"
        f"If John flies to Toronto, his sister will pick him up at the airport.\n"
        f"If Alice is invited, her daughter will bring her to the party.\n\n"
        f"Now complete the sentence logically based on the structure above."
    )

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant generating structured sentences."},
            {"role": "user", "content": prompt},
        ],
        max_tokens=50,
        temperature=0.7,
    )


    sentence = response.choices[0].message.content.strip()
    return sentence


def main():
    file_path = "/content/drive/MyDrive/Colab Notebooks/Type5.csv"
    data = pd.read_csv(file_path)


    data['vocabulary'] = data['vocabulary'].astype(str)
    data['cmp1'] = data['cmp1'].fillna('0').astype(str)
    subjects = data[data['subjects'] == 1]['vocabulary'].tolist()
    verb1_cmp1 = data[['vocabulary', 'cmp1']].dropna()

    sentences_generated = []


    while len(sentences_generated) < 400:
      subject = random.choice(subjects)

        # randomly select a verb and complement
        row = verb1_cmp1.sample().iloc[0]
        verb1 = row['vocabulary']
        cmp1_values = row['cmp1'].split(', ') if row['cmp1'] != "0" else ["0"]

        cmp1 = random.choice(cmp1_values).strip()

        # generate second clause dynamically
        second_clause = "his sister will pick him up at the airport."

        try:
            sentence = generate_sentence_with_gpt(subject, verb1, cmp1, second_clause)
            if sentence not in sentences_generated:  # this is to avoid duplicates
                sentences_generated.append(sentence)
                print(f"Sentence: {sentence}")
        except Exception as e:
            print(f"Error generating sentence for {subject}, {verb1}, {cmp1}: {e}")
            continue

    output_file = "/content/type5-generated-sentences.csv"
    pd.DataFrame({"Sentence": sentences_generated}).to_csv(output_file, index=False)
    print(f"Sentences saved to {output_file}")

if __name__ == "__main__":
    main()


Sentence: If Sarah attends the conference, her colleague will meet her at the hotel.
Sentence: If Sarah passes her driving test, her best friend will take her out for a celebratory dinner.
Sentence: If Sarah finishes her work early, her husband will take her out for dinner.
Sentence: If David finishes his work early, his colleagues will take him out for lunch.
Sentence: If Sarah passes her driving test, her parents will buy her a car.
Sentence: If Sarah's car breaks down, her coworker will give her a ride to the office.
Sentence: If Tom passes his driving test, his parents will buy him a car.
Sentence: If Sarah takes the train to the city, her friend will meet her at the station.
Sentence: If David finishes his meeting early, his colleague will drive him to the conference.
Sentence: If Sarah finishes her presentation early, her colleagues will take her out for lunch.
Sentence: If Sarah has a doctor's appointment, her husband will drive her to the clinic.
Sentence: If Sarah lands in New